# Embeddings

In [53]:
import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from natsort import natsorted
import os
from PIL import Image
from scipy.stats import pearsonr
import json
from torchvision import transforms

In [54]:
# EMB_DIR = '../datasets/embeddings'
TRAIN_DIR = '../datasets/train_big'
VAL_DIR = '../datasets/val_big'

device = torch.device("mps")

resnext = torchvision.models.resnext101_32x8d(torchvision.models.ResNeXt101_32X8D_Weights.IMAGENET1K_V2).to(device)
resnet = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT).to(device)

/Users/lucasgomez/opt/miniconda3/envs/bashlab_cogenv/lib/python3.8/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [55]:
def read_data(path, size):
    transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                            ])

    imgs = []
    cats = []

    i = 0
    for trial_fp in os.listdir(path):
        if 'trial' not in trial_fp:
            continue

        trial_fp = os.path.join(path, trial_fp)
        
        for fp in os.listdir(trial_fp):
            fp = os.path.join(trial_fp, fp)
            
            if fp[-5:] == '2.png':
                img = Image.open(fp)
                img = transform(img)
                imgs.append(img)
            elif 'trial_info' in fp:
                cat = json.load(open(fp))['objects'][-1]['category']
                cats.append(cat)
        i += 1
        if i == size:
            return imgs, cats
    return imgs, cats

imgs_train, labels_train = read_data(TRAIN_DIR, 5000)
imgs_val, labels_val = read_data(VAL_DIR, 500)

In [56]:
class ImageDataset(Dataset):
    def __init__(self, imgs):
        self.imgs = imgs
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        return self.imgs[idx].to(device)
    
trainDL = DataLoader(ImageDataset(imgs_train), batch_size=128, shuffle=False)
valDL = DataLoader(ImageDataset(imgs_val), batch_size=128, shuffle=False)

In [80]:
resnext.eval()
resnet.eval()

def img_encoder(model, images):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    
    model.avgpool.register_forward_hook(get_activation('layer'))

    with torch.no_grad():
        embs = model(images)
        embs = torch.squeeze(activation['layer'])
    return embs

In [82]:
train_embs = []
val_embs = []

for t_images in trainDL:
    train_embs.append(img_encoder(resnet, t_images))

for v_images in valDL:
    val_embs.append(img_encoder(resnet, v_images))

In [85]:
train_embs = torch.cat(tuple(train_embs)).cpu()
val_embs = torch.cat(tuple(val_embs)).cpu()

# RDMs

In [47]:
train_embs = img_encoder(resnext, train_objects)

def construct_RDM(activations):
	num_images = len(activations)
	print(num_images)
	RDM = np.zeros((num_images, num_images))

	for x in range(num_images):
		for y in range(num_images):
			if x<=y:
				correl = 1 - (pearsonr(activations[x].cpu().detach().numpy().flatten(), activations[y].cpu().detach().numpy().flatten()))[0]
				RDM[x][y] = correl
				RDM[y][x] = correl
	return RDM.astype(float)

rdm = construct_RDM(train_embs)

NameError: name 'train_objects' is not defined

In [48]:
rdm.shape

NameError: name 'rdm' is not defined

In [ ]:
import matplotlib.pyplot as plt

labels = [cat + ' ' + str(x) for cat in cats for x in [1,2,3] ]

fig, ax = plt.subplots(figsize=rdm.shape)
cax = ax.matshow(rdm, interpolation='nearest')
ax.grid(False)
plt.xticks(range(rdm.shape[0]), labels, rotation=90)
plt.yticks(range(rdm.shape[0]), labels)
fig.colorbar(cax)
plt.show()


# Classification

In [86]:

#batchxchannelxWxH. take mean over W and H dims to get batchxchannel in the output
train_embs.shape

torch.Size([5000, 2048])

In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression(fit_intercept=True,
                        multi_class='auto',
                        penalty='l2', #ridge regression
                        solver='saga',
                        max_iter=10000,
                        C=50)

clf.fit(train_embs, labels_train)

LogisticRegression(C=50, max_iter=10000, solver='saga')

In [88]:
print('Train acc:', clf.score(train_embs, labels_train))
print('Val acc:', clf.score(val_embs, labels_val))

Train acc: 1.0
Val acc: 0.948


In [89]:
predictions = clf.predict(val_embs)

cm = metrics.confusion_matrix(y_true=labels_val, 
                         y_pred = predictions, 
                        labels = clf.classes_)
cm

array([[75,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 60,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 65,  0,  0,  0,  0,  4],
       [ 0,  2,  0, 57,  1,  0,  0,  0],
       [ 0,  0,  0,  0, 59,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 54,  0,  0],
       [ 0,  2,  1,  0,  0,  0, 55,  0],
       [ 2,  0,  3, 10,  0,  0,  1, 49]])

In [90]:
# !pip -q install seaborn
import seaborn as sns

plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, 
            linewidths=.5, square = True, cmap = 'Blues_r');

plt.ylabel('Actual label')
plt.xlabel('Predicted label')
all_sample_title = 'Accuracy Score: {0}'.format(clf.score(val_X, val_y))
plt.title(all_sample_title);


NameError: name 'plt' is not defined

In [ ]:
# from sklearn.svm import LinearSVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_classification




# clf = make_pipeline(StandardScaler(), LinearSVC(dual="auto", random_state=0, tol=1e-5))
# clf.fit(X, y)
# print(clf.named_steps['linearsvc'].coef_)
# print(clf.named_steps['linearsvc'].intercept_)
